In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Use BeautifulSoup to scrap web table

In [2]:
source = requests.get('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
#print(table.prettify())
ca_df = pd.read_html(str(table))[0]

To clean the table as per required

In [11]:
ca_df = ca_df[ca_df.Borough != 'Not assigned']
ca_df = ca_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
ca_df['Neighbourhood'] = ca_df['Neighbourhood'].replace(to_replace = 'Not assigned', value = ca_df['Borough'])
ca_df.rename(columns = {"Postcode":"PostalCode"}, inplace=True)
ca_df.head()

PostalCode      Borough                         Neighbourhood
0        M1B  Scarborough                         Rouge,Malvern
1        M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2        M1E  Scarborough       Guildwood,Morningside,West Hill
3        M1G  Scarborough                                Woburn
4        M1H  Scarborough                             Cedarbrae

Get geographical coordinates for each postal code

In [12]:
ca_coord_df = pd.read_csv("https://cocl.us/Geospatial_data")
ca_coord_df.rename(columns = {"Postal Code":"PostalCode"}, inplace=True)
ca_coord_df.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [6]:
ca_coord_df.shape

(103, 3)

In [13]:
ca_df_final = pd.merge(ca_df, ca_coord_df, on='PostalCode')
ca_df_final.head()

PostalCode      Borough                         Neighbourhood   Latitude  \
0        M1B  Scarborough                         Rouge,Malvern  43.806686   
1        M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2        M1E  Scarborough       Guildwood,Morningside,West Hill  43.763573   
3        M1G  Scarborough                                Woburn  43.770992   
4        M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [14]:
ca_df_final.shape

(103, 5)